# Fast Predictive Distributions with SKI+LOVE

## Overview

In this notebook, we'll give a brief tutorial on how to use Lanczos Variance Estimates (LOVE) to achieve fast predictive distributions as described in this paper https://arxiv.org/abs/1803.06058. To see LOVE in use with exact GPs, see `fast_variances_exact_(LOVE).ipynb`. For a tutorial on using the fast sampling mechanism described in the paper, see `fast_sampling_ski_(LOVE).ipynb`.

LOVE is an algorithm for approximating the predictive covariances of a Gaussian process in constant time after linear time precomputation. In this notebook, we will train a deep kernel learning model with SKI on one of the UCI datasets used in the paper, and then compare the time required to make predictions with each model.

**NOTE**: The timing results reported in the paper compare the time required to compute (co)variances __only__. Because excluding the mean computations from the timing results requires hacking the internals of GPyTorch, the timing results presented in this notebook include the time required to compute predictive means, which are not accelerated by LOVE. Nevertheless, as we will see, LOVE achieves impressive speed-ups.

In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
from torch import nn, optim
from gpytorch.kernels import RBFKernel, ScaleKernel, GridInterpolationKernel
from gpytorch.means import ConstantMean
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.random_variables import GaussianRandomVariable

# Make plots inline
%matplotlib inline

## Loading Data

For this example notebook, we'll be using the `elevators` UCI dataset used in the paper. Running the next cell downloads a copy of the dataset that has already been scaled and normalized appropriately. For this notebook, we'll simply be splitting the data using the first 80% of the data as training and the last 20% as testing.

**Note**: Running the next cell will attempt to download a ~400 KB dataset file to the current directory.

In [2]:
import urllib.request
import os.path
from scipy.io import loadmat
from math import floor

if not os.path.isfile('elevators.mat'):
    print('Downloading \'elevators\' UCI dataset...')
    urllib.request.urlretrieve('https://drive.google.com/uc?export=download&id=1jhWL3YUHvXIaftia4qeAyDwVxo6j1alk', 'elevators.mat')
    
data = torch.Tensor(loadmat('elevators.mat')['data'])
X = data[:, :-1]
X = X - X.min(0)[0]
X = 2 * (X / X.max(0)[0]) - 1
y = data[:, -1]

# Use the first 80% of the data for training, and the last 20% for testing.
train_n = int(floor(0.8*len(X)))

train_x = X[:train_n, :].contiguous().cuda()
train_y = y[:train_n].contiguous().cuda()

test_x = X[train_n:, :].contiguous().cuda()
test_y = y[train_n:].contiguous().cuda()

## Defining the DKL Feature Extractor

Next, we define the deep feature extractor we'll be using for DKL. In this case, we use a fully connected network with the architecture `d -> 1000 -> 500 -> 50 -> 2`, as described in the original DKL paper. All of the code below uses standard PyTorch implementations of neural network layers.

In [3]:
data_dim = train_x.size(-1)

class LargeFeatureExtractor(nn.Sequential):           
    def __init__(self):                                      
        super(LargeFeatureExtractor, self).__init__()        
        self.add_module('linear1', nn.Linear(data_dim, 1000))
        self.add_module('relu1', nn.ReLU())                  
        self.add_module('linear2', nn.Linear(1000, 500))     
        self.add_module('relu2', nn.ReLU())                  
        self.add_module('linear3', nn.Linear(500, 50))       
        self.add_module('relu3', nn.ReLU())                  
        self.add_module('linear4', nn.Linear(50, 2))         
                                                             
feature_extractor = LargeFeatureExtractor().cuda()

## Defining the GP Model

We now define the GP model. For more details on the use of GP models, see our simpler examples. This model uses a `GridInterpolationKernel` (SKI) with an RBF base kernel. The forward method passes the input data `x` through the neural network feature extractor defined above, scales the resulting features to be between 0 and 1, and then calls the kernel.

In [4]:
class GPRegressionModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood):
            super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
            self.mean_module = ConstantMean()
            self.base_covar_module = ScaleKernel(RBFKernel())
            self.covar_module = GridInterpolationKernel(self.base_covar_module, grid_size=100,
                                                        grid_bounds=[(-1.1, 1.1), (-1.1, 1.1)])
            self.feature_extractor = feature_extractor

        def forward(self, x):
            projected_x = self.feature_extractor(x)
            projected_x = projected_x - projected_x.min(0)[0]
            projected_x = 2 * (projected_x / projected_x.max(0)[0]) - 1
            mean_x = self.mean_module(projected_x)
            covar_x = self.covar_module(projected_x)
            return GaussianRandomVariable(mean_x, covar_x)

In [5]:
likelihood = GaussianLikelihood().cuda()
model = GPRegressionModel(train_x, train_y, likelihood).cuda()

## Training the model

The cell below trains the DKL model above, finding optimal hyperparameters using Type-II MLE. We run 20 iterations of training using the `Adam` optimizer built in to PyTorch. With a decent GPU, this should only take a few seconds.

In [6]:
# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam([
    {'params': model.parameters()},  # Includes GaussianLikelihood parameters
], lr=0.1)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iterations = 20
def train():
    for i in range(training_iterations):
        # Zero backprop gradients
        optimizer.zero_grad()
        # Get output from model
        output = model(train_x)
        # Calc loss and backprop derivatives
        loss = -mll(output, train_y)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f' % (i + 1,
                                           training_iterations,
                                           loss.item()))
        optimizer.step()
        
# See dkl_mnist.ipynb for explanation of this flag
with gpytorch.settings.use_toeplitz(False):
    %time train()

Iter 1/20 - Loss: 0.948
Iter 2/20 - Loss: 0.892
Iter 3/20 - Loss: 0.842
Iter 4/20 - Loss: 0.794
Iter 5/20 - Loss: 0.746
Iter 6/20 - Loss: 0.698
Iter 7/20 - Loss: 0.647
Iter 8/20 - Loss: 0.597
Iter 9/20 - Loss: 0.549
Iter 10/20 - Loss: 0.504
Iter 11/20 - Loss: 0.456
Iter 12/20 - Loss: 0.406
Iter 13/20 - Loss: 0.356
Iter 14/20 - Loss: 0.304
Iter 15/20 - Loss: 0.254
Iter 16/20 - Loss: 0.208
Iter 17/20 - Loss: 0.158
Iter 18/20 - Loss: 0.113
Iter 19/20 - Loss: 0.061
Iter 20/20 - Loss: 0.015
CPU times: user 4.25 s, sys: 1.22 s, total: 5.47 s
Wall time: 5.46 s


## Make Predictions using Standard SKI Code

The next cell gets the predictive covariance for the test set (and also technically gets the predictive mean, stored in `preds.mean()`) using the standard SKI testing code, with no acceleration or precomputation. 

**Note:** Full predictive covariance matrices (and the computations needed to get them) can be quite memory intensive. Depending on the memory available on your GPU, you may need to reduce the size of the test set for the code below to run. If you run out of memory, try replacing `test_x` below with something like `test_x[:1000]` to use the first 1000 test points only, and then restart the notebook.

In [7]:
import time

# Set into eval mode
model.eval()
likelihood.eval()
with torch.no_grad(), gpytorch.settings.use_toeplitz(False):
    start_time = time.time()
    preds = model(test_x[:1000])
    exact_covar = preds.covar().evaluate()
    exact_covar_time = time.time() - start_time

In [8]:
print('Time to compute exact mean + covariances: {:.2f}s'.format(exact_covar_time))

Time to compute exact mean + covariances: 1.53s


## Clear Memory and any Precomputed Values

The next cell clears as much as possible to avoid influencing the timing results of the fast predictive variances code. Strictly speaking, the timing results above and the timing results to follow should be run in entirely separate notebooks. However,  this will suffice for this simple example.

In [9]:
# Clear as much 'stuff' as possible
import gc
gc.collect()
torch.cuda.empty_cache()
model.train()
likelihood.train()

GaussianLikelihood()

## Compute Predictions with LOVE, but Before Precomputation

Next we compute predictive covariances (and the predictive means) for LOVE, but starting from scratch. That is, we don't yet have access to the precomputed cache discussed in the paper. This should still be faster than the full covariance computation code above.

In this simple example, we allow a rank 10 root decomposition, although increasing this to rank 20-40 should not affect the timing results substantially.

In [10]:
# Set into eval mode
model.eval()
likelihood.eval()
with torch.no_grad(), gpytorch.settings.use_toeplitz(False), gpytorch.beta_features.fast_pred_var(), gpytorch.settings.max_root_decomposition_size(10):
    start_time = time.time()
    preds = model(test_x[:1000])
    fast_time_no_cache = time.time() - start_time

## Compute Predictions with LOVE After Precomputation

The above cell additionally computed the caches required to get fast predictions. From this point onwards, unless we put the model back in training mode, predictions should be extremely fast. The cell below re-runs the above code, but takes full advantage of both the mean cache and the LOVE cache for variances.

In [11]:
with torch.no_grad(), gpytorch.settings.use_toeplitz(False), gpytorch.beta_features.fast_pred_var(), gpytorch.settings.max_root_decomposition_size(10):
    start_time = time.time()
    preds = model(test_x[:1000])
    fast_covar = preds.covar().evaluate()
    fast_time_with_cache = time.time() - start_time

In [12]:
print('Time to compute mean + covariances (no cache) {:.2f}s'.format(fast_time_no_cache))
print('Time to compute mean + variances (cache): {:.2f}s'.format(fast_time_with_cache))

Time to compute mean + covariances (no cache) 0.42s
Time to compute mean + variances (cache): 0.05s


## Compute Error between Exact and Fast Variances

Finally, we compute the mean absolute error between the fast variances computed by LOVE (stored in fast_covar), and the exact variances computed previously. 

Note that these tests were run with a root decomposition of rank 10, which is about the minimum you would realistically ever run with. Despite this, the fast variance estimates are quite good. If more accuracy was needed, increasing `max_root_decomposition_size` to 30 or 40 would provide even better estimates.

In [13]:
print('MAE between exact covar matrix and fast covar matrix: {}'.format((exact_covar - fast_covar).abs().mean()))

MAE between exact covar matrix and fast covar matrix: 0.0002387298591202125
